In [ ]:
%load_ext autoreload
%autoreload 2

In [39]:
from utils.DBUtils import *
DBUpdater().update()


# Data Formatting

In [ ]:
from urllib.parse import urljoin
from dotenv import load_dotenv
from utils.DataUtils import *
from datetime import *
from bson import ObjectId
import pandas as pd
import numpy as np
import requests
import random
import torch
import json
import os
load_dotenv()

In [41]:
username = os.getenv("mongodb_user")
password = os.getenv("mongodb_password")

In [42]:
from modules.database_tables import Database
from modules.database_tables.Orders import Orders
from modules.database_tables.Products import Products
from modules.database_tables.TaskCards import TaskCards
from modules.database_tables.ProductOperations import Operations

In [43]:
class GSHubAPI:
    def __init__(self) -> None:
        self.domain: str = "https://premo.gshub.nl/"
    
    def learn_url(self, days_in_past: int) -> str:
        return urljoin(self.domain, f"api/dinand/learn/{days_in_past}")
    
    @property
    def play_url(self) -> str:
        return urljoin(self.domain, "api/dinand/play/")

In [44]:
play_url = GSHubAPI().play_url

In [45]:
db = Database(username, password)
orders = Orders(username, password)
products = Products(username, password)
taskcards = TaskCards(username, password)
operations = Operations(username, password)

In [46]:
def safe_to_datetime(x):
    try:
        return pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S.%fZ')
    except (ValueError, TypeError):
        return x

In [47]:
def safe_to_int(x):
    try:
        if isinstance(x, ObjectId):
            return int(str(x), 16)
        return int(x)
    except (ValueError, TypeError):
        return x

In [48]:
data = json_from_database(orders, products, taskcards, operations)

In [ ]:
df = flatten_orders(data)
df.head()

In [ ]:
df['time_util_delivery'] = df['delivery_date'] - df['start_at']
df['time_util_delivery'] = df['time_util_delivery'].dt.total_seconds() / 3600
df['duration_since_order_created'] = df['start_at'] - df['order_created_at']
df['duration_since_order_created'] = df['duration_since_order_created'].dt.total_seconds() / 3600
df.head()

# Model